In [5]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np
import argparse
import facenet
import detect_face
import os
import sys
import math
import pickle
from sklearn.svm import SVC

In [6]:
with tf.Graph().as_default():

    with tf.Session() as sess:

        datadir = './aligned/'
        dataset = facenet.get_dataset(datadir)
        paths, labels = facenet.get_image_paths_and_labels(dataset)
        print('Number of classes: %d' % len(dataset))
        print('Number of images: %d' % len(paths))

        print('Loading feature extraction model')
        modeldir = './modelSaved/20170511-185253/20170511-185253.pb'
        facenet.load_model(modeldir)

        images_placeholder = tf.get_default_graph().get_tensor_by_name("input:0")
        embeddings = tf.get_default_graph().get_tensor_by_name("embeddings:0")
        phase_train_placeholder = tf.get_default_graph().get_tensor_by_name("phase_train:0")
        embedding_size = embeddings.get_shape()[1]

class['deemarc', 'unknown']
Number of classes: 2
Number of images: 50
Loading feature extraction model
Model filename: ./modelSaved/20170511-185253/20170511-185253.pb


In [7]:
print(embeddings.get_shape())
print(images_placeholder.get_shape)

(?, 128)
<bound method Tensor.get_shape of <tf.Tensor 'input:0' shape=<unknown> dtype=float32>>


In [8]:
# Run forward pass to calculate embeddings
print('Calculating features for images')
batch_size = 1000
image_size = 160
nrof_images = len(paths)
nrof_batches_per_epoch = int(math.ceil(1.0 * nrof_images / batch_size))
emb_array = np.zeros((nrof_images, embedding_size))
temStr = 'number of image per epoch ' + str(nrof_batches_per_epoch)
print(temStr)

Calculating features for images
number of image per epoch 1


In [9]:
print(nrof_images)
start_index = 1 * batch_size
print(start_index)
end_index = min((1 + 1) * batch_size, nrof_images)
print(end_index)
paths_batch = paths[start_index:end_index]
print(paths_batch)

50
1000
50
[]


In [10]:
emb_array = np.zeros((nrof_images, embedding_size))
for i in range(nrof_batches_per_epoch):
    start_index = i * batch_size
    print('start_index: ' + str (start_index)) 
    end_index = min((i + 1) * batch_size, nrof_images)
    print('end_index: ' + str (end_index)) 
    paths_batch = paths[start_index:end_index]
    images = facenet.load_data(paths_batch, False, False, image_size)
    feed_dict = {images_placeholder: images, phase_train_placeholder: False}
    emb_array[start_index:end_index, :] = sess.run(embeddings, feed_dict=feed_dict)

start_index: 0
end_index: 50


In [11]:
print(emb_array)

[[-0.01490071  0.03965852 -0.02916748 ...,  0.04769536  0.03011187
   0.13832133]
 [-0.03563273  0.05278063 -0.04849524 ..., -0.09301965 -0.01848567
   0.00091949]
 [ 0.00065643  0.00284608 -0.08650736 ...,  0.05988909  0.04742663
   0.06801361]
 ..., 
 [ 0.01656157  0.01339139  0.0325558  ...,  0.05930727  0.03481149  0.09068   ]
 [-0.12511477 -0.04967868 -0.02054418 ...,  0.0076438  -0.04555624
   0.08097502]
 [ 0.10240729 -0.06520001 -0.14631735 ...,  0.07053272 -0.0563504
  -0.00889783]]


In [12]:
print(emb_array[0])

[-0.01490071  0.03965852 -0.02916748 -0.07327384  0.03740805 -0.12118302
  0.01806211 -0.19123268 -0.09963465  0.01315908  0.06709067  0.08580038
 -0.05757007  0.02856414 -0.04124936  0.0325281  -0.12710212 -0.04057883
  0.07477987 -0.0648637  -0.19588596  0.08105141  0.01804628 -0.00426263
 -0.03238119 -0.1068694  -0.13389543  0.09163025  0.01435086  0.0420995
 -0.09084287 -0.03761289 -0.06202683 -0.07141073 -0.01976791 -0.14572039
  0.02195624 -0.01534773  0.02358442  0.0792337   0.16750066 -0.03042983
  0.0313882  -0.07581261  0.04505441 -0.0166865  -0.00464379  0.05038027
  0.01745155  0.07473124 -0.04480956 -0.0433794  -0.04011886  0.01781305
 -0.13802072 -0.09721706 -0.01702882 -0.06366867 -0.07362789 -0.01402358
 -0.08387844  0.0656831   0.03144043 -0.02313725  0.02673012 -0.05280549
  0.0748082   0.01934608 -0.0516469   0.012383   -0.22707348 -0.07394178
  0.08447153  0.14238741  0.11705635 -0.0049876  -0.05457335 -0.09084585
  0.0566313  -0.1299372  -0.12606598  0.02478188 -0.

In [13]:
print(labels)
print(len(labels))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
50


In [14]:
# KNN Classifier  
def knn_classifier(train_x, train_y):  
    from sklearn.neighbors import KNeighborsClassifier  
    model = KNeighborsClassifier(n_neighbors=3)  
    model.fit(train_x, train_y)  
    return model  

In [15]:
from sklearn.naive_bayes import GaussianNB
def gaussianNB_classifier(train_x, train_y): 
    model = GaussianNB()
    model.fit(train_x, train_y)  
    return model

In [16]:
from sklearn.svm import SVC
def svc_classifier(train_x, train_y): 
    model = SVC(kernel='linear', probability=True)
    model.fit(train_x, train_y)  
    return model

In [20]:
class eucliDist_clssifier():
    def __init__(self,track):
        self.target = track
        
    def distance(self, v1, v2):
        return np.sqrt(np.sum((v1 - v2) ** 2))   
    
    def predict(self,Persons):
        matched = []
        for curPer in Persons:
            dist = self.distance(self.target,curPer)
            print(dist)
            if dist > 1:
                matched.append(1)
            else:
                matched.append(0)
#                 print('matched')
        
        return matched
        

In [21]:
from sklearn import metrics  

classifier_filename = './class/my_classifier.pkl'
classifier_filename_exp = os.path.expanduser(classifier_filename)

# Train classifier
print('Training classifier')
# model = SVC(kernel='linear', probability=True)
# classifiers = svc_classifier 
model = eucliDist_clssifier(emb_array[0])
# model = classifiers(emb_array,labels) 
predict = model.predict(emb_array) 
print(predict)
# model.fit(emb_array, labels)
accuracy = metrics.accuracy_score(labels, predict)  
print ('accuracy: %.2f%%' % (100 * accuracy)  ) 
# # Create a list of class names
class_names = [cls.name.replace('_', ' ') for cls in dataset]

# # Saving classifier model
with open(classifier_filename_exp, 'wb') as outfile:
    pickle.dump((model, class_names), outfile)
print('Saved classifier model to file "%s"' % classifier_filename_exp)
print('Goodluck')

Training classifier
0.0
0.893975289497
0.639209917284
0.856836591821
0.803008827245
0.713504419447
0.738244478486
1.20447515905
0.886609528465
1.15744942522
0.664108734812
0.65334866366
0.771409151189
0.721613037629
0.693516848423
0.764705454229
0.72247081489
0.723692799107
0.757012508455
0.8283088661
0.905530889778
0.954708491738
0.769823333134
0.66577300779
0.744211226317
0.823391849894
1.01094820464
0.927382096494
0.791898857214
1.08941943974
1.27077548709
1.2662202292
1.33741188193
1.38516535285
1.23937142043
1.31179414292
1.41189175579
1.35118569113
1.08630635925
0.99767290034
1.12554251262
1.46072342688
1.42572335476
1.18859693068
1.18211845562
1.49811774429
1.10265227733
1.26200548789
1.34976038231
1.38637353497
[0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
accuracy: 92.00%
Saved classifier model to file "./class/my_classifier.pkl"
Goodluck


In [ ]:
print(predict)
print(labels)

In [ ]:
predict = model.predict_proba(emb_array) 
print(predict)

In [ ]:
print(labels)
type(labels)

In [ ]:
import numpy as np
def distance(v1, v2):
    return np.sqrt(np.sum((v1 - v2) ** 2))    

In [ ]:
type(emb_array)

In [ ]:
import numpy as np

sucpect = emb_array[0]
ret = distance(sucpect,emb_array)
predict = []
for person in emb_array:
    dist = distance(person,sucpect)
    print(dist)
    if dist > 1:
        predict.append(1)
    else:
        predict.append(0)
print(predict)

In [ ]:
accuracy = metrics.accuracy_score(labels, predict)  

In [ ]:
print(accuracy)